In [1]:
import pandas as pd
from pandas.io import sql

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('dat-test.db')

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [4]:
data = pd.read_csv('../../../lesson-15/assets/dataset/rossmann.csv', low_memory=False)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
data.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)

In [6]:
sql.read_sql('select * from rossmann_sales limit 10', con=conn)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


> #### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [7]:
# TODO

> #### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [8]:
# TODO

> #### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [9]:
# TODO

> #### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [10]:
# TODO

#### Exercises

1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

In [8]:
walmart_sales = pd.read_csv('../../assets/dataset/walmart-sales.csv')
print walmart_sales.head()

walmart_feat = pd.read_csv('../../assets/dataset/features.csv')
print walmart_feat.head()

   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        39.93       2.514        NaN        NaN   
3      1  2010-02-26        46.63       2.561        NaN        NaN   
4      1  2010-03-05        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN        NaN  211.242170         8.106       True  
2        NaN        NaN        NaN  211

In [15]:
connW = sqlite3.connect('wm.db')

In [16]:
walmart_sales.to_sql('wm_sales',
            con=connW,
            if_exists='replace',
            index=False)
walmart_feat.to_sql('wm_feat',
            con=connW,
            if_exists='replace',
            index=False)

In [19]:
sql.read_sql('select Store, Date, Fuel_Price from wm_feat where Temperature > 90 limit 10', con=connW)

,Store,Date,Fuel_Price
0,1,2011-08-05,3.684
1,1,2011-08-12,3.638
2,2,2011-07-29,3.682
3,2,2011-08-05,3.684
4,2,2011-08-12,3.638
5,2,2011-08-26,3.523
6,2,2012-08-03,3.417
7,5,2011-08-05,3.684
8,5,2011-08-12,3.638
9,5,2011-09-02,3.533


In [22]:
#Select the store, date and weekly sales and temperature
sql.read_sql("""
select a.Store, a.Date, a.Weekly_Sales, b.Temperature from wm_sales a 
join wm_feat b
on a.Store = b.Store and a.Date = b.Date
limit 10
""", con=connW)

,Store,Date,Weekly_Sales,Temperature
0,1,2010-02-05,24924.50,42.31
1,1,2010-02-12,46039.49,38.51
2,1,2010-02-19,41595.55,39.93
3,1,2010-02-26,19403.54,46.63
4,1,2010-03-05,21827.90,46.50
5,1,2010-03-12,21043.39,57.79
6,1,2010-03-19,22136.64,54.58
7,1,2010-03-26,26229.21,51.45
8,1,2010-04-02,57258.43,62.27
9,1,2010-04-09,42960.91,65.86


In [24]:
#What were average sales on holiday vs. non-holiday sales
sql.read_sql("""
select AVG(Weekly_Sales), IsHoliday from wm_sales
group by IsHoliday
""", con=connW)

,AVG(Weekly_Sales),IsHoliday
0,15901.445069,0
1,17035.823187,1


In [29]:
#What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees
sql.read_sql("""
select AVG(a.Weekly_Sales), a.IsHoliday from wm_sales a 
join wm_feat b
on a.Store = b.Store and a.Date = b.Date
where b.Temperature < 32
group by a.IsHoliday
""", con=connW)

,AVG(a.Weekly_Sales),IsHoliday
0,15275.770307,0
1,15111.471238,1
